# CNN for Text Classification

In [1]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
NUM_CLASSES = 10
VOCAB_SIZE = 


SyntaxError: invalid syntax (<ipython-input-3-c32d9fb3d3f7>, line 3)

In [4]:
# Basic model parameters as external flags.
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_float('momentum', 0.8, 'momentum hyperparameters')
flags.DEFINE_integer('max_steps', 1000, 'Number of steps to run trainer.')
flags.DEFINE_integer('batch_size', 128, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_string('log_dir', 'logs_txt', 'Directory to put the log data.')
flags.DEFINE_float('dropout', 0.5, 'dropout ratio')
flags.DEFINE_float('L2', 0.01, 'Ridge Regularization')
flags.DEFINE_float('L1', 0.01, 'LASSO Regularization')
flags.DEFINE_integer('embedding_size', 10, 'Size of embedding vector')
flags.DEFINE_integer('filter_size', 10, 'Size of embedding vector')
flags.DEFINE_integer('num_filters', 4, 'Number of feature maps')

In [5]:
def placeholder_inputs(batch_size?, seq_size, num_classes):
    """Generate placeholder variables to represent the input tensors.
    
    Args:
    batch_size: The batch size.?
    seq_size: The sequence size.
    num_classes: The Number of classes
    
    Returns:
    input_x: text sequence placeholder.
    target_y: Labels placeholder.
    """
    input_x = tf.placeholder(tf.float32, [None, seq_size], name="input_x")
    target_y = tf.placeholder(tf.float32, [None, num_classes], name="target_y") # 1-hot-encoded?

    return input_x, num_classes

SyntaxError: invalid syntax (<ipython-input-5-a934cef8c033>, line 1)

In [2]:
def _inference(input_X, vocab_size, embedding_size, num_filters):
    
        
    def weight_variable(shape, name):
        initial = tf.truncated_normal(shape, stddev=1.0 / math.sqrt(float(shape[0])))
        # initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(shape, name):
        #initial = tf.zeros(shape)
        initial = tf.constant(0.1, shape=shape) # prevent dead neural on initialization
        return tf.Variable(initial, name=name)
    
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID') # maintain layer size

    def max_pool(x, seq_size, filter_size):
        return tf.nn.max_pool(x, ksize=[1, sequence_length - filter_size + 1, 1, 1],
                              strides=[1, 1, 1, 1], padding='VALID')
    
    # Embedding layer
    with tf.name_scope('embedding'):
        initial = tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0)
        W = tf.Variable(initial, name="W")
        # perform parallel lookups of embedding
        embedded_chars = tf.nn.embedding_lookup(W, input_X)
        # Add one dimension
        embedded_chars_expan = tf.expand_dims(embedded_chars, dim=-1)
    
    pooled_outputs = []
    for i, filter_size in enumerate(filter_size_list):
        
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = weight_variable(filter_shape, "W")
            b = bias_variable([num_filters], "b")
            conv = conv2d(embedded_chars_expan, W)
            h = tf.nn.relu6(tf.nn.bias_add(conv, b), name="relu6")
            pool_out = max_pool(x, seq_size, filter_size)
            pool_outputs.append(pool_out)
    
    # Combining pooled features
    num_filter_total = num_filters * len(filter_sizes)
    h_pool = tf.concat(3, pool_outputs)
    h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
    
    # Dropout layer
    with tf.name_scope("dropout"):
        h_drop = tf.nn.dropout(h_pool_flat, FLAGS.dropout)
        
    with tf.name_scope("output"):
        W = tf.get_variable("W",
                            shape = [num_filters_total, num_classes],
                            initializer = tf.contrib.layers.xavier_initializer())
        b = bias_variable([num_classes], "b")
        l2_loss += tf.nn.l2_loss(W) + tf.nn.l2_loss(b)
        scores = tf.nn.xw_plus_b(h_drop, W, b, name = "scores")
        predictions = tf.argmax(score, 1, name = "predictions")
        
    #mean cross-entropy loss
    with tf.name_scope("loss"):
        xentropy = tf.nn.softmax_cross_entropy_with_logits(scores, target_y, name = "cross-entropy")
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
    
    # Accuracy
    with tf.name_scope("accuracy"):
        correct_pred = tf.equal(predictions, tf.argmax(target_y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"), name = "accuracy")
    

In [ ]:

class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

